A different way to do this problem is to use cuts, train a bunch of binary classifiers and then feed them into linalg or scipy optimize.  Note this is not optimized; I just slapped it together for learning! 

https://www.kaggle.com/scirpus/cut-and-run/notebook

In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

`DataFrame.insert(loc, column, value, allow_duplicates=False)`

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train['target'] = np.log1p(train['target'])
test.insert(1,'target',-1.0)

In [3]:
floatcolumns = []
intcolumns = []

for c in train.columns[2:]:
    s = train[c].dtype
    if(s=='float64'):
        floatcolumns.append(c)
    else:
        intcolumns.append(c)

In [4]:
alldata = pd.concat([train,test])
del train,test
gc.collect()

0

In [5]:
alldata['sumofzeros'] = (alldata[intcolumns]==0).sum(axis=1)
train = alldata[alldata.target!=-1]
test = alldata[alldata.target==-1]
del alldata
gc.collect()

28

Quantile-based discretization:

In [6]:
x = pd.qcut(train.target, 7, labels=[0, 1, 2, 3, 4, 5, 6])

In [7]:
for a in [0, 1, 2, 3, 4, 5, 6]:
    print("label:{} range: {} ~ {}".format(a,train.target[x==a].min(),train.target[x==a].max()))

label:0 range: 10.308985993422082 ~ 12.345838935721968
label:1 range: 12.354496960983985 ~ 13.592368256649284
label:2 range: 13.596111238258136 ~ 14.403297778421795
label:3 range: 14.408469567642987 ~ 15.068273812173887
label:4 range: 15.072076098936854 ~ 15.618320027115164
label:5 range: 15.620515418416021 ~ 16.40327466837995
label:6 range: 16.405777798910098 ~ 17.50439003707821


General questions:
- min_child_weight meaning?
- max_depth why could be set to -1?

In [8]:
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    "learning_rate": 0.01,
    "feature_fraction": .5,
    "bagging_fraction": .5,
    #'bagging_freq': 4,
    "max_depth": -1,
    "num_leaves": 100,
    "reg_alpha": 0.3,
    "reg_lambda": 0.1,
    "min_child_weight":10,
    'zero_as_missing':True
}

In [9]:
feats = train.columns[2:]

Use the block below - kernels are slow so I am giving you the "best" params

In [10]:
bestparams = {}
lgb_models = []
for i in [0, 1, 2, 3, 4, 5]:
    print('Start LGB model {} :'.format(i))
    lgtrain = lgbm.Dataset(train[feats],x>i)
    lgb_cv = lgbm.cv(
        params = lgbm_params,
        train_set = lgtrain,
        num_boost_round=2000,
        stratified=False,
        nfold = 5,
        verbose_eval=100,
        seed = 42,
        early_stopping_rounds=75)
    
    lgb_models.append(lgb_cv)
    optimal_rounds = np.argmin(lgb_cv['binary_logloss-mean'])
    best_cv_score = min(lgb_cv['binary_logloss-mean'])
    bestparams[i] = (optimal_rounds,best_cv_score)
    
    del lgtrain
    gc.collect()


Start LGB model 0 :
[100]	cv_agg's binary_logloss: 0.409078 + 0.00434919
[200]	cv_agg's binary_logloss: 0.343841 + 0.00740159
[300]	cv_agg's binary_logloss: 0.327712 + 0.00967397
[400]	cv_agg's binary_logloss: 0.325592 + 0.0114419
Start LGB model 1 :
[100]	cv_agg's binary_logloss: 0.537729 + 0.00665311
[200]	cv_agg's binary_logloss: 0.500689 + 0.0105139
[300]	cv_agg's binary_logloss: 0.490717 + 0.0128444
[400]	cv_agg's binary_logloss: 0.490166 + 0.0151963
Start LGB model 2 :
[100]	cv_agg's binary_logloss: 0.599443 + 0.00354913
[200]	cv_agg's binary_logloss: 0.573178 + 0.00699068
[300]	cv_agg's binary_logloss: 0.565735 + 0.00921225
[400]	cv_agg's binary_logloss: 0.564888 + 0.010625
Start LGB model 3 :
[100]	cv_agg's binary_logloss: 0.606256 + 0.00441442
[200]	cv_agg's binary_logloss: 0.581638 + 0.00621337
[300]	cv_agg's binary_logloss: 0.575195 + 0.00738429
[400]	cv_agg's binary_logloss: 0.574007 + 0.00832521
Start LGB model 4 :
[100]	cv_agg's binary_logloss: 0.54545 + 0.00188235
[200]	

In [11]:
bestparams

{0: (387, 0.32549223081879874),
 1: (363, 0.4896434782858353),
 2: (389, 0.5646998803516646),
 3: (412, 0.5738770523457458),
 4: (371, 0.5016286734255448),
 5: (433, 0.3353076747094824)}

Train with given best parameters for number of steps:

In [12]:
folds = KFold(n_splits=5, shuffle=True, random_state=8668)
oof_preds = np.zeros((train.shape[0],6))
sub_preds = np.zeros((test.shape[0],6))

In [13]:
for i in [0, 1, 2, 3, 4, 5]:
    optimal_rounds, best_cv_score = bestparams[i]
    print("Start Model {} with params {}, {} :".format(i, optimal_rounds, best_cv_score))
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train)):
        print("Fold {}".format(n_fold))
        trn_x, trn_y = train[feats].iloc[trn_idx], x[trn_idx]>i
        val_x, val_y = train[feats].iloc[val_idx], x[val_idx]>i
        
        clf = lgbm.train(lgbm_params,
                         train_set = lgbm.Dataset(trn_x,trn_y),
                         valid_sets = lgbm.Dataset(val_x,val_y),
                         num_boost_round = optimal_rounds + 1,
                         verbose_eval=200)

        oof_preds[val_idx,i] = clf.predict(val_x, num_iteration=optimal_rounds + 1)
        sub_preds[:,i] += clf.predict(test[feats], num_iteration=optimal_rounds + 1) / folds.n_splits

        del clf
        del trn_x, trn_y, val_x, val_y
        gc.collect()

Start Model 0 with params 387, 0.32549223081879874 :
Fold 0
[200]	valid_0's binary_logloss: 0.340278
Fold 1
[200]	valid_0's binary_logloss: 0.338457
Fold 2
[200]	valid_0's binary_logloss: 0.351486
Fold 3
[200]	valid_0's binary_logloss: 0.315322
Fold 4
[200]	valid_0's binary_logloss: 0.362386
Start Model 1 with params 363, 0.4896434782858353 :
Fold 0
[200]	valid_0's binary_logloss: 0.486414
Fold 1
[200]	valid_0's binary_logloss: 0.495586
Fold 2
[200]	valid_0's binary_logloss: 0.512938
Fold 3
[200]	valid_0's binary_logloss: 0.501193
Fold 4
[200]	valid_0's binary_logloss: 0.474956
Start Model 2 with params 389, 0.5646998803516646 :
Fold 0
[200]	valid_0's binary_logloss: 0.554603
Fold 1
[200]	valid_0's binary_logloss: 0.558169
Fold 2
[200]	valid_0's binary_logloss: 0.580341
Fold 3
[200]	valid_0's binary_logloss: 0.572303
Fold 4
[200]	valid_0's binary_logloss: 0.549632
Start Model 3 with params 412, 0.5738770523457458 :
Fold 0
[200]	valid_0's binary_logloss: 0.569519
[400]	valid_0's binary_

In [14]:
off_preds_withbias = np.hstack([oof_preds,np.ones(shape=(oof_preds.shape[0],1))])
sub_preds_withbias = np.hstack([sub_preds,np.ones(shape=(sub_preds.shape[0],1))])

In [15]:
# for validation set
pd.DataFrame(off_preds_withbias).head()

,0,1,2,3,4,5,6
0,0.955381,0.928161,0.681111,0.222671,0.130019,0.036149,1.0
1,0.966263,0.835979,0.580033,0.130780,0.098927,0.029068,1.0
2,0.921431,0.810885,0.725055,0.691308,0.338809,0.124334,1.0
3,0.909655,0.732198,0.576642,0.351515,0.282206,0.091151,1.0
4,0.951261,0.908978,0.722071,0.566446,0.275572,0.079777,1.0


In [16]:
# for test set
pd.DataFrame(sub_preds_withbias).head()

,0,1,2,3,4,5,6
0,0.880080,0.688157,0.426635,0.309395,0.280014,0.111254,1.0
1,0.868473,0.673920,0.548743,0.356798,0.243292,0.108166,1.0
2,0.908824,0.607314,0.700101,0.483961,0.275328,0.099588,1.0
3,0.955987,0.887864,0.862819,0.662578,0.547929,0.496419,1.0
4,0.895302,0.765471,0.649119,0.518052,0.346987,0.187197,1.0


In [17]:
params = np.linalg.lstsq(off_preds_withbias, train.target, rcond=None)

In [18]:
trainpreds = np.dot(off_preds_withbias,params[0])
print(np.sqrt(mean_squared_error(train.target,trainpreds)))

1.397079680635988


In [19]:
testpreds = np.dot(sub_preds_withbias,params[0])
sub = pd.DataFrame({'ID':test.ID, 'target':np.expm1(testpreds)})
sub.to_csv('../output/sub_qcut.csv',index=False)

This model gives 1.86 and 1.43 on Leader board, based on different input random seeds. Thus suffers from high variability.